In [2]:
import numpy as np

In [3]:
from bokeh.plotting import figure, show
from bokeh.layouts import row
from bokeh.models import NumeralTickFormatter
import bokeh.io

bokeh.io.output_notebook()


def plot(input_file):
    file_name = input_file.split("IBMObjectStoreTrace")[1]
    xr, yr = np.loadtxt('./data/'+ file_name + '_read_iat_cdf.csv', delimiter=',', unpack=True)
    xw, yw = np.loadtxt('./data/'+ file_name + '_write_iat_cdf.csv', delimiter=',', unpack=True)
    rawx, rawy = np.loadtxt('./data/'+ file_name + '_raw_cdf.csv', delimiter=',', unpack=True)

    # Reduce data points if more than 500
    if len(xr) > 500:
        xr = xr[::len(xr)//500]
        yr = yr[::len(yr)//500]
    if len(xw) > 500:
        xw = xw[::len(xw)//500]
        yw = yw[::len(yw)//500]
    if len(rawx) > 500:
        rawx = rawx[::len(rawx)//500]
        rawy = rawy[::len(rawy)//500]
    
    # Interpolate to 500 points if less
    if len(xr) < 500:
        xr = np.interp(np.linspace(0, 1, 500), np.linspace(0, 1, len(xr)), xr)
        yr = np.interp(np.linspace(0, 1, 500), np.linspace(0, 1, len(yr)), yr)
    if len(xw) < 500:
        xw = np.interp(np.linspace(0, 1, 500), np.linspace(0, 1, len(xw)), xw)
        yw = np.interp(np.linspace(0, 1, 500), np.linspace(0, 1, len(yw)), yw)
    if len(rawx) < 500:
        rawx = np.interp(np.linspace(0, 1, 500), np.linspace(0, 1, len(rawx)), rawx)
        rawy = np.interp(np.linspace(0, 1, 500), np.linspace(0, 1, len(rawy)), rawy)

    p1 = figure(width=300, height=300, background_fill_color="#fafafa", title=f"GET iat cdf ({file_name})", x_axis_label='GET_iat', y_axis_label='cdf')
    p1.line(xr, yr, line_width=2)
    p2 = figure(width=300, height=300, background_fill_color="#fafafa", title=f"PUT iat cdf ({file_name})", x_axis_label='PUT_iat', y_axis_label='cdf')
    p2.line(xw, yw, line_width=2, color='orange')
    p3 = figure(width=300, height=300, background_fill_color="#fafafa", title=f"GET after PUT times cdf ({file_name})", x_axis_label='rawd', y_axis_label='cdf')
    p3.line(rawx, rawy, line_width=2, color='green')

    p1.xaxis[0].formatter = NumeralTickFormatter(format='0a')
    p2.xaxis[0].formatter = NumeralTickFormatter(format='0a')
    p3.xaxis[0].formatter = NumeralTickFormatter(format='0a')

    # grid = gridplot([[p1], [p2], [p3]])

    show(row(p1, p2, p3))


Loading BokehJS ...

In [20]:
plot("IBMObjectStoreTrace003Part0")

In [31]:
import os
from bokeh.plotting import figure, show
from bokeh.models import NumeralTickFormatter
import bokeh.io
import numpy as np

bokeh.io.output_notebook()

def group_plot(input_files):
    get_iat_data = []
    put_iat_data = []
    get_after_put_data = []
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

    for i, input_file in enumerate(input_files):
        file_name = input_file.split("IBMObjectStoreTrace")[1]
        if i == 0:
            file_name_1 = input_file.split("IBMObjectStoreTrace")[1]
        try:
            xr, yr = np.loadtxt(os.path.join('./data', file_name + '_read_iat_cdf.csv'), delimiter=',', unpack=True)
            xw, yw = np.loadtxt(os.path.join('./data', file_name + '_write_iat_cdf.csv'), delimiter=',', unpack=True)
            rawx, rawy = np.loadtxt(os.path.join('./data', file_name + '_raw_cdf.csv'), delimiter=',', unpack=True)

            # Reduce data points if more than 500
            if len(xr) > 500:
                xr = xr[::len(xr)//500]
                yr = yr[::len(yr)//500]
            if len(xw) > 500:
                xw = xw[::len(xw)//500]
                yw = yw[::len(yw)//500]
            if len(rawx) > 500:
                rawx = rawx[::len(rawx)//500]
                rawy = rawy[::len(rawy)//500]
            get_iat_data.append((xr, yr, file_name))
            put_iat_data.append((xw, yw, file_name))
            get_after_put_data.append((rawx, rawy, file_name))
        except (FileNotFoundError, ValueError, IndexError):
            # print(f"Skipped bad trace: {file_name}")
            # print(f"Error processing file: {file_name}")
            continue

    p1 = figure(width=300, height=300, background_fill_color="#fafafa", title=f"{file_name_1.split('Part')[0]} to {file_name.split('Part')[0]}", x_axis_label='GET iat', y_axis_label='cdf')
    p2 = figure(width=300, height=300, background_fill_color="#fafafa", title=f"{file_name_1.split('Part')[0]} to {file_name.split('Part')[0]}", x_axis_label='PUT iat', y_axis_label='cdf')
    p3 = figure(width=300, height=300, background_fill_color="#fafafa", title=f"{file_name_1.split('Part')[0]} to {file_name.split('Part')[0]}", x_axis_label='GET after PUT times', y_axis_label='cdf')


    for i, (xr, yr, label) in enumerate(get_iat_data):
        color_index = i % len(colors)  
        p1.line(xr, yr, line_width=2, legend_label=label, color=colors[color_index])
    for i, (xw, yw, label) in enumerate(put_iat_data):
        color_index = i % len(colors)  
        p2.line(xw, yw, line_width=2, legend_label=label, color=colors[color_index])
    for i, (rawx, rawy, label) in enumerate(get_after_put_data):
        color_index = i % len(colors)  
        p3.line(rawx, rawy, line_width=2, legend_label=label, color=colors[color_index])


    p1.xaxis[0].formatter = NumeralTickFormatter(format='0a')
    p2.xaxis[0].formatter = NumeralTickFormatter(format='0a')
    p3.xaxis[0].formatter = NumeralTickFormatter(format='0a')

    p1.legend.label_text_font_size = '8pt'
    p1.legend.location = "bottom_right"
    p1.legend.click_policy = "hide"
    p1.legend.background_fill_alpha = 0.5
    p1.legend.spacing = -8 
    p2.legend.label_text_font_size = '8pt'
    p2.legend.location = "bottom_right"
    p2.legend.click_policy = "hide"
    p2.legend.background_fill_alpha = 0.5
    p2.legend.spacing = -8 
    p3.legend.label_text_font_size = '8pt'
    p3.legend.location = "bottom_right"
    p3.legend.click_policy = "hide"
    p3.legend.background_fill_alpha = 0.5
    p3.legend.spacing =  -8 

    show(row(p1, p2, p3))


Loading BokehJS ...

In [32]:
input_files = [f"IBMObjectStoreTrace{str(i).zfill(3)}Part{j}" for i in range(21) for j in range(10)]
group_plot(input_files)

